# BAAI/bge-reranker-large

In [73]:
import os
from transformers import pipeline
import torch
from transformers import BertTokenizer
import json
from transformers import AutoModel
from transformers import AutoTokenizer

# cache_dir = "/Users/yangye/models"
# 设置环境变量
# os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

# 检查环境变量是否设置成功
print(os.environ["HF_ENDPOINT"])
# 设置 CUDA_VISIBLE_DEVICES 环境变量
os.environ["CUDA_VISIBLE_DEVICES"] = "0"



if torch.cuda.is_available():
    print(f"Number of GPUs available: {torch.cuda.device_count()}")
    print("CUDA version:", torch.version.cuda)
    print("Device name:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available. Check your installation and try again.")


https://hf-mirror.com
Number of GPUs available: 1
CUDA version: 12.1
Device name: NVIDIA RTX A6000


In [74]:
classifier = pipeline("sentiment-analysis", model="lxyuan/distilbert-base-multilingual-cased-sentiments-student")

In [75]:
checkpoint= "lxyuan/distilbert-base-multilingual-cased-sentiments-student"
tokenizer = AutoTokenizer.from_pretrained(checkpoint,device=0)

In [76]:
raw_inputs = [
    "我弄丢了我心爱的小狗.",
    "我吃到了我喜爱的牛肉。",
    "我的脚趾砸伤了。",
    "我踩到狗屎了。"
]
inputs=tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

 

{'input_ids': tensor([[ 101, 3976, 3705, 2097, 2146, 3976, 3792, 5383, 5718, 3459, 5429,  119,
          102],
        [ 101, 3976, 2767, 2555, 2146, 3976, 2921, 5383, 5718, 5400, 6501, 1882,
          102],
        [ 101, 3976, 5718, 6558, 7545, 5842, 2231, 2146, 1882,  102,    0,    0,
            0],
        [ 101, 3976, 7569, 2555, 5429, 3492, 2146, 1882,  102,    0,    0,    0,
            0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])}


In [77]:
checkpoint= "lxyuan/distilbert-base-multilingual-cased-sentiments-student"

In [78]:
model = AutoModel.from_pretrained(checkpoint)

In [79]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([4, 13, 768])


In [80]:
from transformers import AutoModelForSequenceClassification
checkpoint= "lxyuan/distilbert-base-multilingual-cased-sentiments-student"
model=AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [81]:
outputs = model(**inputs)

In [82]:
print(outputs.logits.shape)

torch.Size([4, 3])


In [83]:
print(outputs.logits)

tensor([[ 0.5801, -0.7363,  0.1747],
        [ 2.5168, -0.6055, -2.0339],
        [-0.9866, -0.2937,  1.5766],
        [-1.3302, -0.5551,  2.2151]], grad_fn=<AddmmBackward0>)


In [84]:
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

In [85]:
print(predictions)

tensor([[0.5169, 0.1386, 0.3446],
        [0.9482, 0.0418, 0.0100],
        [0.0626, 0.1251, 0.8123],
        [0.0264, 0.0574, 0.9162]], grad_fn=<SoftmaxBackward0>)


In [87]:
model.config.id2label

{0: 'positive', 1: 'neutral', 2: 'negative'}